In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep
import pandas as pd

In [2]:
version = "1.0.0"

# Get the article list

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [4]:
def gedjournalarticles(start_url, journaltitle, filename ):
    # open old file
    olddf = pd.DataFrame({'url': [], 'journal': [] })
    try:
        olddf = pd.read_csv(filename, sep="|")
        print(olddf.tail())
    except:
        pass
    
    driver.get(start_url)
    print(driver.current_url)
    
    # itterate over the journal searching to get the Article list
    articles = []
    def pagefind():
        # get all of the Article on this page
        content = driver.page_source
        if len(content) < 20000:
            print(content)
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # found the Article
        for h5 in soup.findAll("div", {'class':['article-content']}):
            # get link 
            for a in h5.findAll("a", {'class':['title-link']}):        
                articles.append("https://www.mdpi.com"+a['href'])
    
        # close the cooke notification
        try:
            for acookie in driver.find_element_by_id("cookie-notification").find_elements_by_tag_name("a"):
                if re.search('Accept',acookie.text ):
                    acookie.click()
                    sleep(1)
                    break
        except:
            pass
    
        # find next page
        def findnext():
            for ulp in driver.find_elements_by_class_name('pageSelect'):
                for lis in ulp.find_elements_by_tag_name('a'):
                    if re.search('chevron_right', lis.text):
                        # go to the next page
                        lis.click()
                        sleep(120) # need long as the have catche things
                        return True
                    
        found = findnext()
        if found:
            return True
        return False

    nextpage = True
    oldarticles = olddf['url']
    newrun = False
    while  nextpage:
        nextpage = pagefind()
        # filter article
        articles = list( set(articles).difference(set(oldarticles) ))
        # we do in branches the Article
        if len(articles) > 1000:
            print("Have not finished yet! Run the workbook agai!")
            newrun = True
            break
        print(driver.current_url, end='\r')
        
    # filer out old articles
    articles = list( set(articles).difference(set(oldarticles) ))
    print(len(articles), "new article found!")
    
    # make df
    df = pd.DataFrame({'url': list(set(articles)), 'journal': journaltitle })
    
    return df, olddf, newrun

# Process new articles

In [5]:
def getarticledetails(df, olddf, filename):
    
    articles = df['url'].values

    titles = [ None for _ in range(len(df))]
    abstracts = [ None for _ in range(len(df))]
    writers = [ None for _ in range(len(df))]
    dates = [ None for _ in range(len(df))]
    dois = [  None for _ in range(len(df)) ]
    keywords = [  None for _ in range(len(df)) ]

    for idx in range(len(articles)):
        # print percentiage of the process
        print( str(np.round(100*idx/len(df),2))+"%" , end='\r')
    
        url = df.iloc[idx]['url']
        try:
            driver.get(url)
            sleep(2)
            page_content = driver.page_source
        except Exception as e:
            continue
    
        page_soup = BeautifulSoup(page_content, features="lxml", from_encoding='utf-8')
        
        # abstract
        abstract = ""
        for abstdiv in page_soup.findAll("div", {'class':['art-abstract']}):
            abstract = abstdiv.get_text().replace("\xa0", " ").replace("\n", "").replace("View Full-Text", '')
        if len(abstract) > 0:
            abstracts[idx] = abstract
        else:
            # if no abstract we ignore the page
            continue

        # title
        titles[idx] = page_soup.find('title').get_text().split("|")[-1]
            
        # writer and publish date
        writter = []
        key = []
        for m in page_soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == "dc.creator":
                    writter.append( m['content'] )
                elif m['name'] == "dc.date":
                    # change to ISO dateformat
                    dates[idx] = m['content']
                elif m['name'] == "dc.identifier":
                    dois[idx] = m['content']
                elif m['name'] == "dc.subject":
                    key.append(m['content'])
        if len(key) > 0:
            keywords[idx] =  "#".join(key)
            
        # itterate over to get the author institutation
        wlinks =  [ None for _ in range(len(writter))]
        for widx in range(len(writter)):
            w = writter[widx]
            for wlink in driver.find_elements_by_class_name('sciprofiles-link__link'):
                if w == wlink.text:
                    # open author page
                    wlinks[widx] =  wlink.get_attribute('href')
        # now go to the user links and find the instituets            
        for widx in range(len(writter)):
            if wlinks[widx] is None:
                continue
            else:
                driver.get(wlinks[widx])
                # get new page source
                usecontent = driver.page_source
                user_soup = BeautifulSoup(usecontent, features="lxml", from_encoding='utf-8')
                
                # get the instituet
                for inst in user_soup.findAll("p", {'class':['profile-widget__contact-info--affiliation']}):
                    writter[widx] = writter[widx] + "--" + inst.get_text().replace("\n", "")
                # get the ircid id
                for orcid in user_soup.findAll("span", {'class':['profile-widget__orcid-val']}):
                    writter[widx] = writter[widx] + "---" + orcid.get_text().replace("\n", "")  
                    
                    
        
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
    
        
    
    # extend the df
    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    
    
    # merge with old df
    df = pd.concat([df, olddf])
    
    # save data
    df.to_csv(filename, sep="|", index=False)
    
    # test
    test = pd.read_csv(filename, sep="|")
    print(test.tail())

# Main

In [6]:
journals = ['Sustainability', 'ISPRS_International_Journal_of_Geo-Information', 'Land' ]
start_urls = [ 
    "https://www.mdpi.com/search?q=&authors=&journal=sustainability&article_type=&search=Search&section=&special_issue=&volume=&issue=&number=&page=",
    "https://www.mdpi.com/search?q=&authors=&journal=ijgi&article_type=&search=Search&section=&special_issue=&volume=&issue=&number=&page=",
    "https://www.mdpi.com/search?q=&authors=&journal=land&article_type=&search=Search&section=&special_issue=&volume=&issue=&number=&page="    
]

In [7]:
# temp filter
journals = ['Sustainability' ]
start_urls = [ 
    "https://www.mdpi.com/search?q=&authors=&journal=sustainability&article_type=&search=Search&section=&special_issue=&volume=&issue=&number=&page=",
]

In [ ]:
for jidx in range(len(journals)):
    filename = filename = 'data/journal_'+ journals[jidx] +'_'+version+'.csv'
    
    # search for articles
    df, olddf, newrun = gedjournalarticles(start_urls[jidx], journals[jidx], filename )
    
    # get the articles details
    getarticledetails(df, olddf, filename)
    
    while newrun:
        df, olddf, newrun = gedjournalarticles(start_urls[jidx], journals[jidx], filename )
        getarticledetails(df, olddf, filename)

                                             url         journal  \
10976  https://www.mdpi.com/2071-1050/12/20/8492  Sustainability   
10977  https://www.mdpi.com/2071-1050/12/20/8363  Sustainability   
10978  https://www.mdpi.com/2071-1050/12/20/8319  Sustainability   
10979  https://www.mdpi.com/2071-1050/12/20/8584  Sustainability   
10980  https://www.mdpi.com/2071-1050/12/20/8500  Sustainability   

                                                   title                 doi  \
10976   A Review of 3D Printing in Construction and i...  10.3390/su12208492   
10977   Saving with a Social Impact: Evidence from Tr...  10.3390/su12208363   
10978   Relationship between Weather Variables and Ne...  10.3390/su12208319   
10979   Determining the Competitiveness of Spa-Center...  10.3390/su12208584   
10980   Does Human Resource Investment for Internal C...  10.3390/su12208500   

                                                abstract  \
10976  Construction industry is very labor-intensi

/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


                                             url         journal  \
11982  https://www.mdpi.com/2071-1050/12/20/8492  Sustainability   
11983  https://www.mdpi.com/2071-1050/12/20/8363  Sustainability   
11984  https://www.mdpi.com/2071-1050/12/20/8319  Sustainability   
11985  https://www.mdpi.com/2071-1050/12/20/8584  Sustainability   
11986  https://www.mdpi.com/2071-1050/12/20/8500  Sustainability   

                                                   title                 doi  \
11982   A Review of 3D Printing in Construction and i...  10.3390/su12208492   
11983   Saving with a Social Impact: Evidence from Tr...  10.3390/su12208363   
11984   Relationship between Weather Variables and Ne...  10.3390/su12208319   
11985   Determining the Competitiveness of Spa-Center...  10.3390/su12208584   
11986   Does Human Resource Investment for Internal C...  10.3390/su12208500   

                                                abstract  \
11982  Construction industry is very labor-intensi

/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


                                             url         journal  \
12983  https://www.mdpi.com/2071-1050/12/20/8492  Sustainability   
12984  https://www.mdpi.com/2071-1050/12/20/8363  Sustainability   
12985  https://www.mdpi.com/2071-1050/12/20/8319  Sustainability   
12986  https://www.mdpi.com/2071-1050/12/20/8584  Sustainability   
12987  https://www.mdpi.com/2071-1050/12/20/8500  Sustainability   

                                                   title                 doi  \
12983   A Review of 3D Printing in Construction and i...  10.3390/su12208492   
12984   Saving with a Social Impact: Evidence from Tr...  10.3390/su12208363   
12985   Relationship between Weather Variables and Ne...  10.3390/su12208319   
12986   Determining the Competitiveness of Spa-Center...  10.3390/su12208584   
12987   Does Human Resource Investment for Internal C...  10.3390/su12208500   

                                                abstract  \
12983  Construction industry is very labor-intensi

...

                                             url         journal  \
14013  https://www.mdpi.com/2071-1050/12/20/8492  Sustainability   
14014  https://www.mdpi.com/2071-1050/12/20/8363  Sustainability   
14015  https://www.mdpi.com/2071-1050/12/20/8319  Sustainability   
14016  https://www.mdpi.com/2071-1050/12/20/8584  Sustainability   
14017  https://www.mdpi.com/2071-1050/12/20/8500  Sustainability   

                                                   title                 doi  \
14013   A Review of 3D Printing in Construction and i...  10.3390/su12208492   
14014   Saving with a Social Impact: Evidence from Tr...  10.3390/su12208363   
14015   Relationship between Weather Variables and Ne...  10.3390/su12208319   
14016   Determining the Competitiveness of Spa-Center...  10.3390/su12208584   
14017   Does Human Resource Investment for Internal C...  10.3390/su12208500   

                                                abstract  \
14013  Construction industry is very labor-intensi

/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


                                             url         journal  \
15037  https://www.mdpi.com/2071-1050/12/20/8492  Sustainability   
15038  https://www.mdpi.com/2071-1050/12/20/8363  Sustainability   
15039  https://www.mdpi.com/2071-1050/12/20/8319  Sustainability   
15040  https://www.mdpi.com/2071-1050/12/20/8584  Sustainability   
15041  https://www.mdpi.com/2071-1050/12/20/8500  Sustainability   

                                                   title                 doi  \
15037   A Review of 3D Printing in Construction and i...  10.3390/su12208492   
15038   Saving with a Social Impact: Evidence from Tr...  10.3390/su12208363   
15039   Relationship between Weather Variables and Ne...  10.3390/su12208319   
15040   Determining the Competitiveness of Spa-Center...  10.3390/su12208584   
15041   Does Human Resource Investment for Internal C...  10.3390/su12208500   

                                                abstract  \
15037  Construction industry is very labor-intensi

/usr/local/lib/python3.8/dist-packages/bs4/__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


                                             url         journal  \
16069  https://www.mdpi.com/2071-1050/12/20/8492  Sustainability   
16070  https://www.mdpi.com/2071-1050/12/20/8363  Sustainability   
16071  https://www.mdpi.com/2071-1050/12/20/8319  Sustainability   
16072  https://www.mdpi.com/2071-1050/12/20/8584  Sustainability   
16073  https://www.mdpi.com/2071-1050/12/20/8500  Sustainability   

                                                   title                 doi  \
16069   A Review of 3D Printing in Construction and i...  10.3390/su12208492   
16070   Saving with a Social Impact: Evidence from Tr...  10.3390/su12208363   
16071   Relationship between Weather Variables and Ne...  10.3390/su12208319   
16072   Determining the Competitiveness of Spa-Center...  10.3390/su12208584   
16073   Does Human Resource Investment for Internal C...  10.3390/su12208500   

                                                abstract  \
16069  Construction industry is very labor-intensi